In [1]:
# 443901, m.joo@wustl.edu, Joo, Michael
# 472926, michael.kincheloe@wustl.edu, Kincheloe, Michael
# 457870, jfeibelman@wustl.edu, Feibelman, Jason
# 463959, m.fishman@wustl.edu, Fishman, Matthew 
# 458013, mattwitzerman@wustl.edu, Witzerman, Matt

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [3]:
# train = pd.read_csv('/kaggle/input/suspicious-transaction-detection/train.csv')
# test = pd.read_csv('/kaggle/input/suspicious-transaction-detection/test.csv')

train = pd.read_csv('../suspicious-transaction-detection/train.csv')

N_train, dim = train.shape
# N_test, _ = test.shape

X_train = train.drop(['Target'], axis=1)
y_train = train['Target'].copy()

In [4]:
N_train

472432

In [5]:
med_features = np.concatenate((['Amount'],
                               ['T_{}'.format(t) for t in range(15)], 
                               ['C_{}'.format(c) for c in range(9, 23)],
                               ['C_26', 'C_27', 'C_28'], 
                               ['V_{}'.format(v) for v in range(339)], 
                               ['O_0', 'O_1', 'O_5', 'O_6', 'O_9', 'O_18', 'O_31', 'O_36'],
                               ['A_0', 'M_1']
                              ))
mod_features = np.concatenate((['C_0', 'C_1', 'C_2', 'C_3', 'C_4', 'C_6', 'C_7', 'C_8', 'C_23'],
                               ['O_2', 'O_3', 'O_10', 'O_11', 'O_12', 'O_15', 'O_20', 'O_21', 'O_22', 'O_24', 'O_26', 'O_27', 'O_28', 'O_29', 'O_30', 'O_32', 'O_33', 'O_34', 'O_35', 'O_37', 'O_38'],
                               ['A_1']
                              ))
oh_features = np.concatenate((['Goods', 'C_5', 'C_24', 'C_25'],
                              ['O_4', 'O_7', 'O_8', 'browser', 'O_16', 'O_17', 'os', 'O_23', 'O_25',  'O_39'],
                              ['E_same', 'M_0']
                             ))

print(len(med_features) + len(mod_features) + len(oh_features))

429


In [6]:
# features = pd.concat([X_train, test]).reset_index(drop=True)
features = X_train.copy()

features = features.drop(['TransactionID', 'Timestamp', 'O_14'], axis=1)

features['browser'] = features['O_13'].str.split(' ').str[0]
features['os'] = features['O_19'].str.split(' ').str[0]
features['E_0'] = features['E_0'].fillna('empty')
features['E_same'] = np.where(features['E_0'] == features['E_1'], 'T', 'F')

features = features.drop(['O_13', 'O_19', 'E_0', 'E_1'], axis=1)

for feature in med_features:
    med = X_train[feature].median()
    features[feature] = features[feature].fillna(med)
for feature in mod_features:
    mod = X_train[feature].mode()[0]
    #print('{}: {}'.format(feature, mod))
    features[feature] = features[feature].fillna(mod)

features_prepared = pd.get_dummies(features, columns=oh_features)

features_prepared.replace('T', 1, inplace=True)
features_prepared.replace('F', 0, inplace=True)

X_train_prepared = features_prepared.iloc[:N_train, :].copy()
# X_test_prepared = features_prepared.iloc[N_train:, :].copy()

print(X_train_prepared.shape)
# print(X_test_prepared.shape)

(472432, 497)


In [ ]:
# After the data filling we perform PCA
from sklearn.decomposition import PCA



With the data prepared, we train a Random Forest classifier model on the training data. To evaluate the model, we apply 5-fold cross validation and observe the mean and standard deviation of the accuracy of our model.

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

forest = RandomForestClassifier()
forest_error = cross_val_score(forest, X_train_prepared, y_train, cv=5)
print(forest_error.mean())
print(forest_error.std())

/Users/matthewfishman/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/matthewfishman/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/matthewfishman/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/matthewfishman/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureW

0.977611592267109
7.118974721401519e-05


The results seem promising. Note that a Random Forest model, by default, returns the predicted label of a data point. Since we want the probability of a transaction being fraudulent instead of its predicted label, we return the number of votes that believe a transaction is fraudulent divided by the total number of votes from the Random Forest model. 

In [8]:
forest.fit(X_train_prepared, y_train)
y_pred = forest.predict_proba(X_test_prepared)
my_submission = pd.DataFrame({'TransactionID': test.TransactionID, 'Target': y_pred[:, 1]})
my_submission.to_csv('submission.csv', index=False)

/Users/matthewfishman/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


NameError: name 'X_test_prepared' is not defined